In [4]:
import time
import requests
import sqlite3
from bs4 import BeautifulSoup

# ===========================
# スクレイピング部
# ===========================
def scrape_ranking_urls(target_url, sleep_sec=1, top_n=20):
    """
    ランキングページ（target_url）から上位top_n件の求人URLを取得して返す。
    """
    response = requests.get(target_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')

    # 例: 「a[href][data-way='39']」をリストに格納
    ranking_items = soup.select('a[href][data-way="39"]')

    ranking_urls = []
    for item in ranking_items:
        url = item.get('href')
        if url:
            # 相対パスの場合の補完
            if url.startswith('/'):
                url = "https://type.jp/" + url
            # 重複チェック
            if url not in ranking_urls:
                ranking_urls.append(url)

    # 先頭 top_n 件だけに絞る
    ranking_urls_top = ranking_urls[:top_n]

    print(f"\n=== {target_url} の上位 {top_n} 件 ===")

    # 順番に表示（確認用）
    for idx, url in enumerate(ranking_urls_top, start=1):
        print(f"{idx}位: {url}")

    # スリープ（サイトに負荷をかけすぎないように）
    time.sleep(sleep_sec)

    return ranking_urls_top

def scrape_job_details(job_url):
    """
    求人詳細ページ（job_url）にアクセスし、
    'カテゴリ', '年収', '企業名', '勤務地', '歓迎スキル', '雇用形態' を取得して返す。
    実際のHTML構造に合わせてCSSセレクタや処理を修正してください。
    """
    response = requests.get(job_url)
    response.raise_for_status()
    soup = BeautifulSoup(response.text, 'html.parser')


    title_element = soup.select_one('#content > div > div > main > header > section > section.head_upr.detail.pull-left.clearfix.mt10.mb0 > h1')
    title_text = title_element.get_text(strip=True) if title_element else None

    salary_element = soup.select_one('#content > div > div > main > header > section > section.head_upr.detail.pull-left.clearfix.mt10.mb0 > ul > li.mt4 > span.ico_salary')
    salary_text = salary_element.get_text(strip=True) if salary_element else None

    company_element = soup.select_one('#content > div > div > main > header > p > a')
    company_text = company_element.get_text(strip=True) if company_element else None

    location_element = soup.select_one('#content > div > div > main > section.bigarea.bosyuarea.mod-job-content > div.box.clearfix.uq-detail-area > div > section.uq-detail-location')
    location_text = location_element.get_text(strip=True) if location_element else None

    welcomed_skill_element = soup.select_one('#content > div > div > main > section.bigarea.bosyuarea.mod-job-content > div.box.clearfix.uq-detail-skill > div > section.uq-detail-welcome > p')
    welcomed_skill_text = welcomed_skill_element.get_text(strip=True) if welcomed_skill_element else None

    employment_type_element = soup.select_one('#content > div > div > main > section.bigarea.bosyuarea.mod-job-content > div.box.clearfix.uq-detail-type > div > p')
    employment_type_text = employment_type_element.get_text(strip=True) if employment_type_element else None

    return {
        'title': title_text,
        'url': job_url,
        '年収': salary_text,
        '企業名': company_text,
        '勤務地': location_text,
        '歓迎スキル': welcomed_skill_text,
        '雇用形態': employment_type_text
    }

# ===========================
# データベース関連部 (SQLite)
# ===========================
def init_db(db_path="jobs.db"):
    """
    SQLiteのDB (jobs.db) が存在しない場合は作成し、
    'jobs' テーブルを必要に応じて作成する（存在する場合はスキップ）。
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    create_table_sql = """
    CREATE TABLE IF NOT EXISTS jobs (
        id INTEGER PRIMARY KEY AUTOINCREMENT,
        title TEXT,
        url TEXT,
        salary TEXT,
        company TEXT,
        location TEXT,
        welcomed_skill TEXT,
        employment_type TEXT
    );
    """
    cursor.execute(create_table_sql)
    conn.commit()
    conn.close()

def insert_data(data_list, db_path="jobs.db"):
    """
    スクレイピングで取得した求人データ(リスト)をSQLiteのjobsテーブルにまとめてINSERTする。
    data_list = [
      {
        'title': ...,
        'url': ...,
        '年収': ...,
        '企業名': ...,
        '勤務地': ...,
        '歓迎スキル': ...,
        '雇用形態': ...
      },
      ...
    ]
    """
    conn = sqlite3.connect(db_path)
    cursor = conn.cursor()

    insert_sql = """
    INSERT INTO jobs
    (title, url, salary, company, location, welcomed_skill, employment_type)
    VALUES (?, ?, ?, ?, ?, ?, ?);
    """

    records = []
    for data in data_list:
        record = (
            data.get('title'),
            data.get('url'),
            data.get('年収'),
            data.get('企業名'),
            data.get('勤務地'),
            data.get('歓迎スキル'),
            data.get('雇用形態')
        )
        records.append(record)

    cursor.executemany(insert_sql, records)
    conn.commit()
    conn.close()

# ===========================
# メイン処理
# ===========================
def main():
    # 1. DB初期化（jobs.dbとjobsテーブルを作成）
    init_db("jobs.db")

    # スクレイピングのベースURL
    base = "https://type.jp/rank/"
    paths = [
        "",
        "development/",
        "pm/",
        "infrastructure/",
        "engineer/",
        "sales/",
        "service/",
        "office/",
        "others/"
    ]

    # 総合URLリストを格納するリスト
    all_scraped_urls = []

    # 2. ランキングURLを取得
    for path in paths:
        if path == "":
            target_url = base
        else:
            target_url = base.rstrip('/') + "/" + path.strip('/') + "/"

        urls_top20 = scrape_ranking_urls(target_url, sleep_sec=1, top_n=20)
        all_scraped_urls.extend(urls_top20)

    print("\n=== まとめて取得した全URL ===")
    for url in all_scraped_urls:
        print(url)

    # 3. 各求人URLから詳細情報を取得
    scraped_data_list = []
    print("\n=== 各URLから情報をスクレイピング ===")
    for job_url in all_scraped_urls:
        try:
            job_info = scrape_job_details(job_url)
            scraped_data_list.append(job_info)
            print(job_info)
            # サイトに負荷をかけすぎないよう、適宜スリープ
            time.sleep(1)
        except Exception as e:
            print(f"Error scraping {job_url}: {e}")

    # 4. 取得したデータをDBへ保存
    insert_data(scraped_data_list, db_path="jobs.db")
    print("\n=== データベースにINSERT完了 ===")

if __name__ == "__main__":
    main()


=== https://type.jp/rank/ の上位 20 件 ===
1位: https://type.jp//job-11/1343474_detail/
2位: https://type.jp//job-7/1343441_detail/
3位: https://type.jp//job-7/1343510_detail/
4位: https://type.jp//job-1/1318111_detail/
5位: https://type.jp//job-1/1186701_detail/
6位: https://type.jp//job-1/1023402_detail/
7位: https://type.jp//job-7/1341480_detail/
8位: https://type.jp//job-7/1343814_detail/
9位: https://type.jp//job-1/1300937_detail/
10位: https://type.jp//job-1/1342818_detail/
11位: https://type.jp//job-10/1343800_detail/
12位: https://type.jp//job-7/1341723_detail/
13位: https://type.jp//job-1/1329008_detail/
14位: https://type.jp//job-7/1312221_detail/
15位: https://type.jp//job-7/1344012_detail/
16位: https://type.jp//job-5/1328403_detail/
17位: https://type.jp//job-1/1276310_detail/
18位: https://type.jp//job-11/1344254_detail/
19位: https://type.jp//job-5/1028379_detail/
20位: https://type.jp//job-7/1279006_detail/

=== https://type.jp/rank/development/ の上位 20 件 ===
1位: https://type.jp//job-1/1318111